In [1]:
import dataset
import tensorflow as tf
import time
import os
from datetime import timedelta
import cv2
import math
import numpy as np
import random
import matplotlib.pyplot as plt

batch_size = 50
classes = ['bul','chil','de','e','ga','gack','gu','gyo','ho','jung','ka','kim','min','nyu','nyun','oh','pal','ryuck','sa','sam','sip','su','wang']
num_classes = len(classes)

validation_size = 0.1
img_size = 32
num_channels = 3
train_path='training_data'

# We shall load all the training and validation images and labels into memory using openCV and use that during training
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)

print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t\t{}".format(len(data.train.labels)))
print("Number of files in Validation-set:\t{}".format(len(data.valid.labels)))

Going to read training images
Now going to read bul files (Index: 0)
Now going to read chil files (Index: 1)
Now going to read de files (Index: 2)
Now going to read e files (Index: 3)
Now going to read ga files (Index: 4)
Now going to read gack files (Index: 5)
Now going to read gu files (Index: 6)
Now going to read gyo files (Index: 7)
Now going to read ho files (Index: 8)
Now going to read jung files (Index: 9)
Now going to read ka files (Index: 10)
Now going to read kim files (Index: 11)
Now going to read min files (Index: 12)
Now going to read nyu files (Index: 13)
Now going to read nyun files (Index: 14)
Now going to read oh files (Index: 15)
Now going to read pal files (Index: 16)
Now going to read ryuck files (Index: 17)
Now going to read sa files (Index: 18)
Now going to read sam files (Index: 19)
Now going to read sip files (Index: 20)
Now going to read su files (Index: 21)
Now going to read wang files (Index: 22)
Complete reading input data. Will Now print a snippet of it
Num

In [2]:
summary = tf.summary.merge_all()
session = tf.Session()
summary_writer = tf.summary.FileWriter('./board/sample_2',session.graph)
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128

In [3]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [4]:
def create_convolutional_layer(input,num_input_channels, conv_filter_size, num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer

In [5]:
def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer

In [6]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [7]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('Cost',cost)
session.run(tf.global_variables_initializer()) 

In [8]:
def show_progress(epoch, feed_dict_train):  #, feed_dict_validate, val_loss
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    #val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}"
    print(msg.format(epoch + 1, acc))

In [9]:
def show_progress2(epoch, feed_dict_train):  #, feed_dict_validate, val_loss
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    #val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Test Epoch {0} --- Test Accuracy: {1:>6.1%}"
    print(msg.format(epoch + 1, acc))

In [10]:
total_iterations = 0
saver = tf.train.Saver()

In [ ]:
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations, total_iterations + num_iteration):
        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)
        avg_cost = 0
        
        feed_dict_tr = {x: x_batch, y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        total_batch = int(data.train.num_examples/batch_size)
        if i % int(data.train.num_examples/batch_size) == 0: 
           # summary_str = session.run(summary, feed_dict = feed_dict_tr)
           # summary_writer.add_summary(summary_str, i)
           # summary_writer.flush()
            #val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            show_progress(epoch, feed_dict_tr) #, feed_dict_val, val_loss
            show_progress2(epoch, feed_dict_val)
            #save_path = saver.save(session, '.\model.ckpt')
            saver.save(session, os.path.join(os.getcwd(), 'chinaModel.ckpt'))
            #saver.save(session, 'dogs-cats-model') 
            
            c, _ = session.run([cost, optimizer], feed_dict=feed_dict_tr)
            avg_cost += c / total_batch

            print('cost =', '{:.9f}'.format(avg_cost))


    total_iterations += num_iteration

train(num_iteration=2000)

Training Epoch 1 --- Training Accuracy:  10.0%
Test Epoch 1 --- Test Accuracy:   2.0%
cost = 0.038076366
Training Epoch 2 --- Training Accuracy:  46.0%
Test Epoch 2 --- Test Accuracy:  30.0%
cost = 0.037353646
Training Epoch 3 --- Training Accuracy:  72.0%
Test Epoch 3 --- Test Accuracy:  64.0%
cost = 0.028526196
Training Epoch 4 --- Training Accuracy:  92.0%
Test Epoch 4 --- Test Accuracy:  92.0%
cost = 0.007458243
Training Epoch 5 --- Training Accuracy:  92.0%
Test Epoch 5 --- Test Accuracy:  92.0%
cost = 0.005077949
Training Epoch 6 --- Training Accuracy:  92.0%
Test Epoch 6 --- Test Accuracy:  84.0%
cost = 0.004171918
Training Epoch 7 --- Training Accuracy:  92.0%
Test Epoch 7 --- Test Accuracy:  78.0%
cost = 0.003668844
Training Epoch 8 --- Training Accuracy:  94.0%
Test Epoch 8 --- Test Accuracy:  84.0%
cost = 0.003315522
Training Epoch 9 --- Training Accuracy:  94.0%
Test Epoch 9 --- Test Accuracy:  94.0%
cost = 0.003021038
Training Epoch 10 --- Training Accuracy:  96.0%
Test Ep

In [ ]:
tf.train.write_graph(session.graph.as_graph_def(), "models/", "graph.pb")

In [ ]:
r = random.randint(0,len(data.valid.labels)-1)
plt.imshow(data.valid.images[r:r + 1].reshape(32, 32, 3),
               cmap='Greys', interpolation='nearest')
plt.show()
tf.train.write_graph(session.graph_def, '.', 'chinaModel.pbtxt')  
print("Label: ", session.run(tf.argmax(data.valid.labels[r:r + 1], 1)))
idx = session.run(tf.argmax(data.valid.labels[r:r + 1], 1))
print("name: ", classes[idx[0]])

print("Prediction: ", session.run(tf.argmax(layer_fc2, 1), feed_dict={x: data.valid.images[r:r + 1]}))